In [1]:
import pandas as pd
import numpy as np
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec

In [2]:
cleaned_data = pd.read_csv('../Datasets/Cleaned-Data.csv')
artificial_features = pd.read_csv('../Datasets/Artificial_Features.csv')

In [3]:
cleaned_data.head()

,question1,question2,is_duplicate,Q1_cleaned,Q2_cleaned
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor koh noor diamond,would happen indian government stole kohinoor ...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection using vpn,internet speed increased hacking dns
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math math divided
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,one dissolve water quikly sugar salt methane c...,fish would survive salt water


In [4]:
cleaned_data.shape

(404287, 5)

In [5]:
cleaned_data.isnull().sum()

question1         0
question2         0
is_duplicate      0
Q1_cleaned      177
Q2_cleaned      175
dtype: int64

In [6]:
cleaned_data.dropna(inplace=True)

In [7]:
artificial_features.head()

,q1_len,q2_len,q1_words,q2_words,common_words,total_words,words_share,token_set,token_sort,partial_token_set,partial_token_sort
0,28,27,4,4,2,8,0.25,71,69,100,75
1,33,34,4,4,2,8,0.25,75,66,100,64
2,83,7,14,2,1,16,0.06,60,13,100,57
3,23,18,4,3,2,7,0.29,76,54,100,50
4,53,84,8,10,0,14,0.00,20,22,26,33


In [8]:
artificial_features.shape

(100000, 11)

In [9]:
# dataset = pd.concat([cleaned_data, artificial_features], axis=1)
# dataset = dataset.sample(100000)
# dataset.shape

In [10]:
# dataset.head()

In [11]:
# cleaned_data = dataset[cleaned_data.columns]
# artificial_features = dataset[artificial_features.columns]
# cleaned_data.shape, artificial_features.shape

In [12]:
cleaned_data = cleaned_data.sample(100000)

In [13]:
cleaned_data.shape

(100000, 5)

# Word2Vec

In [14]:
cleaned_data.shape

(100000, 5)

In [15]:
questions = cleaned_data['Q1_cleaned'].tolist() + cleaned_data['Q2_cleaned'].tolist()

In [16]:
questions = pd.DataFrame(questions, columns=['Questions'])
questions

,Questions
0,get ripped
1,connect simulink plc
2,suggestions ideas improve english writing skills
3,buy moon
4,difference international company multinational...
...,...
199995,b com h student go actuarial science yes statu...
199996,pikachu better starter three others
199997,change default browser
199998,like print something


In [17]:
questions['Questions'].isnull().sum()

0

In [18]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Questions  200000 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB


In [19]:
questions['Tokens'] = [simple_preprocess(str(sentence)) for sentence in questions['Questions']]
questions.head()

,Questions,Tokens
0,get ripped,"[get, ripped]"
1,connect simulink plc,"[connect, simulink, plc]"
2,suggestions ideas improve english writing skills,"[suggestions, ideas, improve, english, writing..."
3,buy moon,"[buy, moon]"
4,difference international company multinational...,"[difference, international, company, multinati..."


In [20]:
vector_size = 500
word2vec_model = Word2Vec(questions['Tokens'], min_count=1, window=10, vector_size=vector_size, workers=8, sg=1)

In [21]:
#word2vec_model.build_vocab(questions['tokens'], progress_per=1000)

In [22]:
#word2vec_model.train(questions['tokens'], total_examples=word2vec_model.corpus_count, epochs=word2vec_model.epochs)

In [23]:
def avgVectors(tokens):
  avg_vector = np.mean([word2vec_model.wv.get_vector(token) for token in tokens], axis=0)
  return avg_vector

In [24]:
questions['avg w2v vectorized'] = questions['Tokens'].apply(avgVectors)

c:\Users\Priyesh Dave\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [25]:
questions.head()

,Questions,Tokens,avg w2v vectorized
0,get ripped,"[get, ripped]","[0.1591037, 0.12783964, 0.19466783, 0.14870681..."
1,connect simulink plc,"[connect, simulink, plc]","[0.0911453, 0.05019818, 0.25945228, 0.03318710..."
2,suggestions ideas improve english writing skills,"[suggestions, ideas, improve, english, writing...","[0.10612717, 0.1330968, 0.20293021, -0.0109047..."
3,buy moon,"[buy, moon]","[0.21124767, -0.03409654, 0.29923323, 0.105853..."
4,difference international company multinational...,"[difference, international, company, multinati...","[0.060116827, 0.13761255, 0.30151713, 0.133050..."


In [26]:
len(questions['avg w2v vectorized'].to_list())

200000

In [27]:
questions['avg w2v vectorized'].shape

(200000,)

In [28]:
middle = int(questions['avg w2v vectorized'].shape[0]/2)
middle

100000

In [29]:
question1 , question2 = questions['avg w2v vectorized'].iloc[0:middle] , questions['avg w2v vectorized'].iloc[middle :]

In [30]:
question1 = question1.reset_index().drop('index', axis=1).rename(columns={'avg w2v vectorized': 'Q1'})

In [31]:
question2 = question2.reset_index().drop('index', axis=1).rename(columns={'avg w2v vectorized': 'Q2'})

In [32]:
question1

,Q1
0,"[0.1591037, 0.12783964, 0.19466783, 0.14870681..."
1,"[0.0911453, 0.05019818, 0.25945228, 0.03318710..."
2,"[0.10612717, 0.1330968, 0.20293021, -0.0109047..."
3,"[0.21124767, -0.03409654, 0.29923323, 0.105853..."
4,"[0.060116827, 0.13761255, 0.30151713, 0.133050..."
...,...
99995,"[-0.0023616066, 0.20501213, 0.28903687, 0.1376..."
99996,"[0.09110853, 0.14864902, 0.24272138, 0.1540661..."
99997,"[0.061042443, 0.079808444, 0.21341662, 0.19023..."
99998,"[0.37136382, 0.047368728, 0.16829772, 0.148179..."


In [33]:
question2

,Q2
0,"[0.1351882, 0.20211737, 0.24466793, 0.18238716..."
1,"[0.18062268, 0.20486212, 0.44039485, 0.2012050..."
2,"[0.16359526, 0.17857885, 0.16626261, -0.018835..."
3,"[0.13385396, 0.045677263, 0.33972287, 0.075053..."
4,"[0.061009254, 0.10987089, 0.3020047, 0.1345269..."
...,...
99995,"[0.13103975, 0.24281748, 0.21392013, 0.1246250..."
99996,"[0.16038403, 0.13329938, 0.22683398, 0.1564481..."
99997,"[0.061042443, 0.079808444, 0.21341662, 0.19023..."
99998,"[0.3154023, 0.24774222, 0.14911468, 0.14163765..."


In [34]:
for index, row in enumerate(question1['Q1']):
  print(index)
  print(row)
  print(type(row.tolist()))
  break

0
[ 1.59103706e-01  1.27839640e-01  1.94667831e-01  1.48706809e-01
 -1.13213390e-01 -1.18471168e-01  8.68537351e-02  2.81193972e-01
 -3.13126519e-02 -7.68457279e-02 -1.31241605e-01  2.24267140e-01
 -4.88885790e-02 -8.71500000e-03  1.62148073e-01 -2.02782676e-01
 -1.30953416e-01 -8.45367908e-02  6.25486393e-03  5.54991961e-02
 -1.15565538e-01 -4.99609523e-02  4.46389243e-02  7.14727677e-03
  5.88550558e-03 -9.60357338e-02  4.94239107e-02 -6.52092993e-02
 -1.08660504e-01  9.59865749e-04  2.93502044e-02 -1.14031985e-01
 -8.09751227e-02 -1.20033368e-01  1.59358665e-01  7.65682533e-02
  7.19376653e-02  1.54070593e-02 -9.00426358e-02 -1.41288012e-01
  1.19292974e-01 -8.14112425e-02 -2.07896262e-01 -4.26040739e-02
  4.32470813e-02 -8.16317648e-02 -1.08839318e-01  3.62626940e-01
 -4.03515659e-02 -6.22265115e-02  6.19551837e-02 -1.00243464e-01
  2.36176133e-01 -1.31385252e-01 -8.66997764e-02 -7.47849941e-02
  1.69160947e-01 -8.57591033e-02 -1.11820489e-01  6.69579655e-02
  1.73209608e-01 -6.135

In [35]:
# Store the vectors for train data in following file
OUTPUT_FOLDER = '../Datasets/'
word2vec_filename = OUTPUT_FOLDER + 'Q1_word2vec.csv'
with open(word2vec_filename, 'w+') as word2vec_file:
    for index, row in enumerate(question1['Q1']):
        #model_vector = (np.mean([word2vec_model.wv.get_vector(token) for token in row['Tokens']], axis=0)).tolist()
        row = row.tolist()
        if index == 0:
            header = ",".join(str(ele) for ele in range(vector_size))
            word2vec_file.write(header)
            word2vec_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(row) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in row] )
        else:
            line1 = ",".join([str(0) for i in range(vector_size)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

In [36]:
Q1 = pd.read_csv('../Datasets/Q1_word2vec.csv')
Q1.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.159104,0.127840,0.194668,0.148707,-0.113213,-0.118471,0.086854,0.281194,-0.031313,-0.076846,...,-0.075193,-0.055711,0.138249,0.016542,0.028278,-0.203409,-0.125087,-0.005672,-0.019439,0.161579
1,0.091145,0.050198,0.259452,0.033187,-0.024802,-0.095558,-0.004581,0.196599,-0.041321,0.037035,...,0.013230,0.009511,0.080315,-0.009877,0.013205,-0.062634,-0.071021,-0.154481,-0.043556,-0.025714
2,0.106127,0.133097,0.202930,-0.010905,-0.051594,-0.147373,0.090241,0.522273,0.116916,0.140777,...,-0.230715,-0.122945,0.039841,-0.003337,0.023143,-0.021244,-0.331358,-0.017265,-0.176439,-0.150574
3,0.211248,-0.034097,0.299233,0.105853,0.037535,-0.040176,-0.121438,0.406516,0.051698,-0.038953,...,0.141510,-0.153866,0.237007,0.126008,-0.259372,-0.040827,-0.048652,-0.297636,-0.159818,-0.002610
4,0.060117,0.137613,0.301517,0.133050,0.014871,-0.153269,0.016563,0.303499,0.085074,0.168118,...,0.070693,-0.061162,0.116656,0.025282,0.142333,-0.014501,-0.282383,-0.173748,0.072248,-0.013023


In [37]:
# Store the vectors for train data in following file
OUTPUT_FOLDER = '../Datasets/'
word2vec_filename = OUTPUT_FOLDER + 'Q2_word2vec.csv'
with open(word2vec_filename, 'w+') as word2vec_file:
    for index, row in enumerate(question2['Q2']):
        #model_vector = (np.mean([word2vec_model.wv.get_vector(token) for token in row['Tokens']], axis=0)).tolist()
        row = row.tolist()
        if index == 0:
            header = ",".join(str(ele + vector_size) for ele in range(vector_size))
            word2vec_file.write(header)
            word2vec_file.write("\n")
        # Check if the line exists else it is vector of zeros
        if type(row) is list:  
            line1 = ",".join( [str(vector_element) for vector_element in row] )
        else:
            line1 = ",".join([str(0) for i in range(vector_size)])
        word2vec_file.write(line1)
        word2vec_file.write('\n')

In [38]:
Q2 = pd.read_csv('../Datasets/Q2_word2vec.csv')
Q2.head()

,500,501,502,503,504,505,506,507,508,509,...,990,991,992,993,994,995,996,997,998,999
0,0.135188,0.202117,0.244668,0.182387,-0.087658,-0.210874,0.099161,0.245719,0.028115,-0.084472,...,-0.098332,-0.118939,0.113934,-0.009238,0.006367,-0.144849,-0.145922,-0.014966,-0.087658,0.115435
1,0.180623,0.204862,0.440395,0.201205,-0.105556,-0.152936,0.054567,0.542428,0.034744,0.035175,...,0.011026,-0.074607,0.178489,0.047545,0.104875,-0.051214,-0.136349,-0.207822,-0.083756,-0.159096
2,0.163595,0.178579,0.166263,-0.018836,-0.115097,-0.127559,0.098684,0.498126,0.114626,0.180547,...,-0.168487,-0.145497,0.000056,0.040941,0.071111,-0.026878,-0.315068,-0.000822,-0.088132,-0.098859
3,0.133854,0.045677,0.339723,0.075053,-0.008764,-0.079616,0.045309,0.443606,0.188744,0.016297,...,0.072816,-0.210135,0.268748,0.064604,-0.109583,0.118173,0.090241,-0.268488,-0.078953,-0.174966
4,0.061009,0.109871,0.302005,0.134527,-0.020455,-0.103552,-0.015302,0.216958,0.050221,0.119446,...,-0.010212,-0.037607,0.105114,0.076840,0.076730,-0.056583,-0.124962,-0.103746,-0.000177,-0.126717


In [39]:
Q1.head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.159104,0.127840,0.194668,0.148707,-0.113213,-0.118471,0.086854,0.281194,-0.031313,-0.076846,...,-0.075193,-0.055711,0.138249,0.016542,0.028278,-0.203409,-0.125087,-0.005672,-0.019439,0.161579
1,0.091145,0.050198,0.259452,0.033187,-0.024802,-0.095558,-0.004581,0.196599,-0.041321,0.037035,...,0.013230,0.009511,0.080315,-0.009877,0.013205,-0.062634,-0.071021,-0.154481,-0.043556,-0.025714
2,0.106127,0.133097,0.202930,-0.010905,-0.051594,-0.147373,0.090241,0.522273,0.116916,0.140777,...,-0.230715,-0.122945,0.039841,-0.003337,0.023143,-0.021244,-0.331358,-0.017265,-0.176439,-0.150574
3,0.211248,-0.034097,0.299233,0.105853,0.037535,-0.040176,-0.121438,0.406516,0.051698,-0.038953,...,0.141510,-0.153866,0.237007,0.126008,-0.259372,-0.040827,-0.048652,-0.297636,-0.159818,-0.002610
4,0.060117,0.137613,0.301517,0.133050,0.014871,-0.153269,0.016563,0.303499,0.085074,0.168118,...,0.070693,-0.061162,0.116656,0.025282,0.142333,-0.014501,-0.282383,-0.173748,0.072248,-0.013023


In [40]:
artificial_features.head()

,q1_len,q2_len,q1_words,q2_words,common_words,total_words,words_share,token_set,token_sort,partial_token_set,partial_token_sort
0,28,27,4,4,2,8,0.25,71,69,100,75
1,33,34,4,4,2,8,0.25,75,66,100,64
2,83,7,14,2,1,16,0.06,60,13,100,57
3,23,18,4,3,2,7,0.29,76,54,100,50
4,53,84,8,10,0,14,0.00,20,22,26,33


In [41]:
dataset_word2vec = pd.concat([Q1,Q2], axis=1)
dataset_word2vec.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.159104,0.127840,0.194668,0.148707,-0.113213,-0.118471,0.086854,0.281194,-0.031313,-0.076846,...,-0.098332,-0.118939,0.113934,-0.009238,0.006367,-0.144849,-0.145922,-0.014966,-0.087658,0.115435
1,0.091145,0.050198,0.259452,0.033187,-0.024802,-0.095558,-0.004581,0.196599,-0.041321,0.037035,...,0.011026,-0.074607,0.178489,0.047545,0.104875,-0.051214,-0.136349,-0.207822,-0.083756,-0.159096
2,0.106127,0.133097,0.202930,-0.010905,-0.051594,-0.147373,0.090241,0.522273,0.116916,0.140777,...,-0.168487,-0.145497,0.000056,0.040941,0.071111,-0.026878,-0.315068,-0.000822,-0.088132,-0.098859
3,0.211248,-0.034097,0.299233,0.105853,0.037535,-0.040176,-0.121438,0.406516,0.051698,-0.038953,...,0.072816,-0.210135,0.268748,0.064604,-0.109583,0.118173,0.090241,-0.268488,-0.078953,-0.174966
4,0.060117,0.137613,0.301517,0.133050,0.014871,-0.153269,0.016563,0.303499,0.085074,0.168118,...,-0.010212,-0.037607,0.105114,0.076840,0.076730,-0.056583,-0.124962,-0.103746,-0.000177,-0.126717


In [42]:
cleaned_data.isnull().sum()

question1       0
question2       0
is_duplicate    0
Q1_cleaned      0
Q2_cleaned      0
dtype: int64

## ANN

In [65]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Embedding, SpatialDropout1D, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, auc, roc_auc_score

from sklearn.model_selection import train_test_split

In [44]:
dataset_word2vec.shape, cleaned_data['is_duplicate'].shape

((100000, 1000), (100000,))

In [50]:
X_train_w2v, X_test_w2v, y_train_w2v, y_test_w2v = train_test_split(dataset_word2vec, cleaned_data['is_duplicate'], test_size=.33, random_state=42, shuffle=True)

In [51]:
y_train_w2v.value_counts()

0    42227
1    24773
Name: is_duplicate, dtype: int64

In [52]:
y_test_w2v.value_counts()

0    20717
1    12283
Name: is_duplicate, dtype: int64

In [53]:
y_train_w2v =  np.array(y_train_w2v).reshape(-1,1)
y_test_w2v =  np.array(y_test_w2v).reshape(-1,1)

In [54]:
X_train_w2v.shape, X_test_w2v.shape, y_train_w2v.shape, y_test_w2v.shape

((67000, 1000), (33000, 1000), (67000, 1), (33000, 1))

In [55]:
X_train_w2v.shape, X_test_w2v.shape, y_train_w2v.shape, y_test_w2v.shape

((67000, 1000), (33000, 1000), (67000, 1), (33000, 1))

In [56]:
X_train_w2v_lstm = np.array(X_train_w2v).reshape(-1, 1, X_train_w2v.shape[1])
X_test_w2v_lstm  = np.array(X_test_w2v).reshape(-1, 1, X_train_w2v.shape[1])
y_train_w2v_lstm = y_train_w2v.reshape(-1, 1, 1)
y_test_w2v_lstm = y_test_w2v.reshape(-1, 1, 1)

In [57]:
X_train_w2v_lstm.shape, X_test_w2v_lstm.shape, y_train_w2v_lstm.shape, y_test_w2v_lstm.shape

((67000, 1, 1000), (33000, 1, 1000), (67000, 1, 1), (33000, 1, 1))

In [62]:
MODEL_NAME = 'W2V_LSTM'

model = Sequential()

model.add(Bidirectional(LSTM(512, input_shape=(1, X_train_w2v.shape[1]), kernel_initializer = 'he_uniform', activation = 'relu', return_sequences=True)))
model.add(Dropout(.3))
model.add(Bidirectional(LSTM(256, input_shape=(1, X_train_w2v.shape[1]), kernel_initializer = 'he_uniform', activation = 'relu', return_sequences=True)))
model.add(Dropout(.3))
model.add(Bidirectional(LSTM(128, kernel_initializer = 'he_uniform', activation='relu')))

model.add(Dense(1, kernel_initializer = 'glorot_uniform', activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [63]:
my_callbacks = [
    EarlyStopping(patience=2),
    ModelCheckpoint(filepath='../LSTM/Callbacks/model.{epoch:02d}-{val_loss:.2f}.h5'),
    TensorBoard(log_dir='../LSTM/logs'),
]

In [64]:
model.fit(X_train_w2v_lstm, y_train_w2v_lstm, validation_data=(X_test_w2v_lstm, y_test_w2v_lstm), epochs=100, batch_size=1024, callbacks=my_callbacks)

Epoch 1/100
66/66 [==============================] - 53s 625ms/step - loss: 0.6087 - accuracy: 0.6784 - val_loss: 0.5480 - val_accuracy: 0.7311
Epoch 2/100
66/66 [==============================] - 36s 541ms/step - loss: 0.5394 - accuracy: 0.7351 - val_loss: 0.5256 - val_accuracy: 0.7421
Epoch 3/100
66/66 [==============================] - 34s 522ms/step - loss: 0.5118 - accuracy: 0.7489 - val_loss: 0.5091 - val_accuracy: 0.7448
Epoch 4/100
66/66 [==============================] - 32s 489ms/step - loss: 0.4924 - accuracy: 0.7572 - val_loss: 0.4928 - val_accuracy: 0.7529
Epoch 5/100
66/66 [==============================] - 32s 493ms/step - loss: 0.4785 - accuracy: 0.7616 - val_loss: 0.4867 - val_accuracy: 0.7544
Epoch 6/100
66/66 [==============================] - 38s 575ms/step - loss: 0.4697 - accuracy: 0.7675 - val_loss: 0.4850 - val_accuracy: 0.7554
Epoch 7/100
66/66 [==============================] - 41s 616ms/step - loss: 0.4601 - accuracy: 0.7719 - val_loss: 0.4792 - val_accuracy:

In [66]:
train_pred_w2v = model.predict(X_train_w2v_lstm)
train_pred_w2v = (train_pred_w2v > 0.5)
train_pred_w2v

test_pred_w2v = model.predict(X_test_w2v_lstm)
test_pred_w2v = (test_pred_w2v > 0.5)
test_pred_w2v

print('Training')
print(accuracy_score(y_train_w2v, train_pred_w2v))
print(classification_report(y_train_w2v, train_pred_w2v))
print(confusion_matrix(y_train_w2v, train_pred_w2v))

print('==================================')

print('Test')
score = accuracy_score(y_test_w2v, test_pred_w2v)
auc_score = roc_auc_score(y_test_w2v, test_pred_w2v)
print('Accuracy Score: ', score)
print('AUC: ', auc_score)
print(classification_report(y_test_w2v, test_pred_w2v))
print(confusion_matrix(y_test_w2v, test_pred_w2v))

Training
0.8005820895522388
              precision    recall  f1-score   support

           0       0.82      0.88      0.85     42227
           1       0.76      0.67      0.71     24773

    accuracy                           0.80     67000
   macro avg       0.79      0.77      0.78     67000
weighted avg       0.80      0.80      0.80     67000

[[37019  5208]
 [ 8153 16620]]
Test
Accuracy Score:  0.7627878787878788
AUC:  0.7327542361686468
              precision    recall  f1-score   support

           0       0.79      0.85      0.82     20717
           1       0.71      0.62      0.66     12283

    accuracy                           0.76     33000
   macro avg       0.75      0.73      0.74     33000
weighted avg       0.76      0.76      0.76     33000

[[17615  3102]
 [ 4726  7557]]


## LSTM

In [72]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import tensorflow

In [67]:
questions.head()

,Questions,Tokens,avg w2v vectorized
0,get ripped,"[get, ripped]","[0.1591037, 0.12783964, 0.19466783, 0.14870681..."
1,connect simulink plc,"[connect, simulink, plc]","[0.0911453, 0.05019818, 0.25945228, 0.03318710..."
2,suggestions ideas improve english writing skills,"[suggestions, ideas, improve, english, writing...","[0.10612717, 0.1330968, 0.20293021, -0.0109047..."
3,buy moon,"[buy, moon]","[0.21124767, -0.03409654, 0.29923323, 0.105853..."
4,difference international company multinational...,"[difference, international, company, multinati...","[0.060116827, 0.13761255, 0.30151713, 0.133050..."


In [68]:
questions.shape

(200000, 3)

In [71]:
voc_size = 16000
embedding_vector_features = 500

In [73]:
# One hot representation of words
onehot_repr=[one_hot(sentence,voc_size) for sentence in questions['Questions']] 
#onehot_repr

In [75]:
onehot_repr[0:5]

[[10704, 15276],
 [3845, 12743, 7833],
 [14629, 3622, 2277, 12660, 6995, 12030],
 [9237, 5204],
 [5437, 690, 15819, 7369, 15819]]

In [76]:
one_hot_dict = dict()

for encodingList, tokensList in zip(onehot_repr, questions['Tokens']):
  for index, word in zip(encodingList, tokensList):
    one_hot_dict[word] = index
    
one_hot_df = pd.DataFrame(one_hot_dict.items(), columns=['Word', 'Encoded Value'])
one_hot_df.head()
one_hot_df.to_csv('../Datasets/One Hot Encoded Data.csv', index=False)

In [83]:
q1_onehot_repr, q2_onehot_repr  = onehot_repr[0: middle] , onehot_repr[middle:]

In [85]:
len(q1_onehot_repr), len(q2_onehot_repr)

(100000, 100000)

In [86]:
# Getting what is the highest count of words in any sentence
q1_max_count = max([len(arr) for arr in q1_onehot_repr])
q1_max_count

53

In [87]:
# Getting what is the highest count of words in any sentence
q2_max_count = max([len(arr) for arr in q2_onehot_repr])
q2_max_count

97

In [88]:
# Embedding the sentences
q1_embedded_docs = pad_sequences(q1_onehot_repr,padding='pre',maxlen=q1_max_count)
print(q1_embedded_docs)

[[    0     0     0 ...     0 10704 15276]
 [    0     0     0 ...  3845 12743  7833]
 [    0     0     0 ... 12660  6995 12030]
 ...
 [    0     0     0 ...  9964  4112 15106]
 [    0     0     0 ...     0     0  7967]
 [    0     0     0 ... 13122 13959 15577]]


In [89]:
# Embedding the sentences
q2_embedded_docs = pad_sequences(q2_onehot_repr,padding='pre',maxlen=q2_max_count)
print(q2_embedded_docs)

[[    0     0     0 ... 10704 15276  2471]
 [    0     0     0 ...  6260  2521 13892]
 [    0     0     0 ... 12030 11212   436]
 ...
 [    0     0     0 ...  9964  4112 15106]
 [    0     0     0 ...   766 12991 11031]
 [    0     0     0 ... 13122 13959 15577]]


In [92]:
len(q1_embedded_docs), len(q2_embedded_docs)

(100000, 100000)

In [93]:
len(q1_embedded_docs[0]), len(q2_embedded_docs[0])

(53, 97)

In [94]:
q1_embedded_docs = np.array(q1_embedded_docs)
q2_embedded_docs = np.array(q2_embedded_docs)

data = np.hstack((q1_embedded_docs, q2_embedded_docs))
embedded_df = pd.DataFrame(data)
embedded_df.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,10704,15276,2471
1,0,0,0,0,0,0,0,0,0,0,...,13560,6857,3598,5763,523,11418,5124,6260,2521,13892
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,2277,12660,6995,12030,11212,436
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,6083,11611,5204
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1320,2082,7369,11848


In [102]:
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=embedded_df.shape[1]))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(200, kernel_initializer  = 'he_uniform', return_sequences=True)))
model.add(Bidirectional(LSTM(100, kernel_initializer  = 'he_uniform')))

model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [96]:
my_callbacks_lstm = [
    EarlyStopping(patience=2),
    ModelCheckpoint(filepath='../LSTM_WordEmbeddings/Callbacks/model.{epoch:02d}-{val_loss:.2f}.h5'),
    TensorBoard(log_dir='../LSTM_WordEmbeddings/logs'),
]

In [98]:
X_LSTM = embedded_df.copy()
y_LSTM = np.array(cleaned_data['is_duplicate'])

In [99]:
X_LSTM.shape, y_LSTM.shape

((100000, 150), (100000,))

In [100]:
X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_LSTM, y_LSTM, test_size=.33, random_state=42, shuffle=True)

In [103]:
model.fit(X_train_lstm, y_train_lstm, validation_data=(X_test_lstm, y_test_lstm), epochs=100, batch_size=1024, callbacks=my_callbacks_lstm)

Epoch 1/100
 8/66 [==>...........................] - ETA: 29:52 - loss: 0.6641 - accuracy: 0.6228